In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from tvDatafeed import TvDatafeed, Interval



In [ ]:


# Login no TradingView
tv = TvDatafeed()

ticker = 'WIN1!'
exchange = 'BMFBOVESPA'


df = tv.get_hist(
    symbol=ticker,
    exchange=exchange,
    interval=Interval.in_daily,
    n_bars=10000
)
df = df[df.index.year >=2000].dropna()
df.index = pd.to_datetime(df.index).normalize().date
df.drop(columns='symbol',inplace=True)
df.index = pd.to_datetime(df.index).normalize()
df['weekday'] = df.index.weekday
df.dropna(inplace=True)
df['ret'] = df['close'].pct_change()
df.tail()




In [ ]:
close = df['close']
open  = df['open']

ma = df["ma"]
m_flag = (close >= ma).astype(int)

# --- sinal 2: retorno mensal comparado ao mês anterior (r_flag) ---
# calcula (close - open) do mês
monthly_diff = (close.resample('ME').last() / open.resample('ME').first()) - 1

# compara com mês anterior
r_flag_monthly = (monthly_diff >= monthly_diff.shift(1)).astype(int)
r_flag = r_flag_monthly.reindex(close.index, method='ffill')

df['m_flag'] = m_flag
df['r_flag'] = r_flag
df.dropna(inplace=True)
df.tail()

In [ ]:
df["position"] = np.where(
    ((df["m_flag"] == 1) & (df["r_flag"] == 1)),
    1,
    0
)

df.head()
df["ret"] = df["Close"].pct_change()
df["strategy_ret"] = df["position"] * df["ret"].shift(-1)

df.dropna(inplace=True)
df["buy_hold"] = df["ret"].cumsum()
df["strategy"] = df["strategy_ret"].cumsum()



In [ ]:
# =========================
# PLOT
# =========================
fig = make_subplots(
    rows=1,
    cols=1,
    shared_xaxes=True,
    specs=[[{"secondary_y": True}]]
)

# Buy & Hold
fig.add_trace(
    go.Scatter(
        x=df.index,
        y=df["buy_hold"] * 100,
        name="Buy & Hold",
        line=dict(width=2)
    ),
    secondary_y=False
)

# Strategy
fig.add_trace(
    go.Scatter(
        x=df.index,
        y=df["strategy"] * 100,
        name="Moving Average Strategy",
        line=dict(width=2)
    ),
    secondary_y=False
)

# Position (0 / 1)
fig.add_trace(
    go.Scatter(
        x=df.index,
        y=df["position"],
        name="Position",
        line=dict(width=1, dash="dot"),
        opacity=0.7
    ),
    secondary_y=True
)

# Layout
fig.update_layout(
    title="SP500 | Dual Momentum",
    xaxis_title="Date",
    yaxis_title="Cumulative Return (%)",
    yaxis2_title="Position",
    legend=dict(x=0.01, y=0.99),
    template="plotly_white",
    height=600
)

# Ajuste do eixo secundário
fig.update_yaxes(range=[-0.05, 1.05], secondary_y=True)

fig.show()



In [ ]:
total_return_bh = df["buy_hold"].iloc[-1]
total_return_strategy = df["strategy"].iloc[-1]

vol_strategy = df["strategy_ret"].std() * np.sqrt(252)
vol_bh = df["ret"].std() * np.sqrt(252)

sharpe_strategy = (
    df["strategy_ret"].mean() / df["strategy_ret"].std()
) * np.sqrt(252)

sharpe_bh = (
    df["ret"].mean() / df["ret"].std()
) * np.sqrt(252)

print("=== RESULTADOS ===")
print(f"Buy & Hold Return: {total_return_bh:.2%}")
print(f"Strategy Return:   {total_return_strategy:.2%}")
print()
print(f"Buy & Hold Vol: {vol_bh:.2%}")
print(f"Strategy Vol:   {vol_strategy:.2%}")
print()
print(f"Buy & Hold Sharpe: {sharpe_bh:.2f}")
print(f"Strategy Sharpe:   {sharpe_strategy:.2f}")
